In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

In [ ]:
spark = SparkSession.builder.appName("diabetes").getOrCreate()

In [ ]:
spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data= spark.read.csv("/content/drive/MyDrive/diabetes.csv", inferSchema=True, header=True)

In [ ]:
data.show(10)

In [ ]:
data.printSchema()

In [ ]:
data.describe().show()

In [ ]:
#pearsonCorr = Correlation.corr(data, 'variety', 'pearson').collect()[0][0]

In [ ]:
data.columns

In [ ]:
assembler = VectorAssembler(inputCols=['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',],
 outputCol='features')

In [ ]:
output = assembler.transform(data)
output

In [ ]:
df_final = output.select('features', 'Outcome')

In [ ]:
df_final.show(10)

In [ ]:
train, test = df_final.randomSplit([0.7, 0.3], seed=42)

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
Logistic_Regression = LogisticRegression(labelCol="Outcome")

In [ ]:
fit = Logistic_Regression.fit(train)

In [ ]:
fit.summary

In [ ]:
summary = fit.summary

In [ ]:
summary.predictions.show()

In [ ]:
summary.predictions.describe().show() 

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
prediction = fit.evaluate(test)

In [ ]:
prediction.predictions.show()

In [ ]:
evaluate = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Outcome")

In [ ]:
accuracy = evaluate.evaluate(prediction.predictions)

In [ ]:
accuracy

In [ ]:
data.createOrReplaceTempView("diabetes")

In [ ]:
sql=spark.sql("SELECT * from diabetes")
sql.show()